[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/search/semantic-search/ner-search/ner-powered-search.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/search/semantic-search/ner-search/ner-powered-search.ipynb)

# NER Powered Semantic Search

**NOT WORKING**: 

ValueError: The argument order for `query()` has changed; please use keyword arguments instead of positional arguments. Example: index.query(vector=[0.1, 0.2, 0.3], top_k=10, namespace='my_namespace')

This notebook shows how to use Named Entity Recognition (NER) for hybrid metadata + vector search with Pinecone. We will:

1. Extract named entities from text.
2. Store them in a Pinecone index as metadata (alongside respective text vectors).
3. We extract named entities from incoming queries and use them to filter and search only through records containing these named entities.

This is particularly helpful if you want to restrict the search score to records that contain information about the named entities that are also found within the query.

Let's get started.

# Install Dependencies

In [31]:
#!pip install sentence_transformers pinecone-client==3.1.0 datasets -qU

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [1]:
import os
from pathlib import Path
from getpass import getpass
from pinecone import Pinecone

pak = "PINECONE_API_KEY"
if pak in os.environ:
    print("Found PINECONE_API_KEY in os.environ, no need to prompt")
else: 
    api_key = getpass("Enter PINECONE API key:")
    os.environ["PINECONE_API_KEY"] = api_key
    
# initialize connection to pinecone (get API key at app.pinecone.io)
# api_key = os.environ.get('PINECONE_API_KEY') or 'PINECONE_API_KEY'

/home/sean/miniforge3/envs/pinecode-haystack-env/lib/python3.10/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


ImportError: cannot import name 'Pinecone' from 'pinecone' (/home/sean/miniforge3/envs/pinecode-haystack-env/lib/python3.10/site-packages/pinecone/__init__.py)

In [33]:
# configure client
pc = Pinecone(api_key=api_key)
idxs = pc.list_indexes
print(idxs)

<bound method Pinecone.list_indexes of <pinecone.control.pinecone.Pinecone object at 0x7f495c3cdfc0>>


# Load and Prepare Dataset

We use a dataset containing ~190K articles scraped from Medium. We select 50K articles from the dataset as indexing all the articles may take some time. This dataset can be loaded from the HuggingFace dataset hub as follows:

In [34]:
from datasets import load_dataset

# load the dataset and convert to pandas dataframe
df = load_dataset(
    "fabiochiu/medium-articles",
    data_files="medium_articles.csv",
    split="train"
).to_pandas()

In [35]:
# drop empty rows and select 20k articles
df = df.dropna().sample(20000, random_state=32)
df.head()

,title,text,url,authors,timestamp,tags
4172,How the Data Stole Christmas,by Anonymous\n\nThe door sprung open and our t...,https://medium.com/data-ops/how-the-data-stole...,[],2019-12-24 13:22:33.143000+00:00,"['Data Science', 'Big Data', 'Dataops', 'Analy..."
174868,Automating Light Switch using the ESP32 Board ...,A story about how I escaped the boring task th...,https://python.plainenglish.io/automating-ligh...,['Tomas Rasymas'],2021-09-14 07:20:52.342000+00:00,"['Programming', 'Python', 'Software Developmen..."
100171,Keep Going Quotes Sayings for When Hope is Lost,It’s a very thrilling thing to achieve a goal....,https://medium.com/@yourselfquotes/keep-going-...,['Yourself Quotes'],2021-01-05 12:13:04.018000+00:00,['Quotes']
141757,When Will the Smoke Clear From Bay Area Skies?,Bay Area cities are contending with some of th...,https://thebolditalic.com/when-will-the-smoke-...,['Matt Charnock'],2020-09-15 22:38:33.924000+00:00,"['Bay Area', 'San Francisco', 'California', 'W..."
183489,"The ABC’s of Sustainability… easy as 1, 2, 3",By Julia DiPrete\n\n(according to the Jackson ...,https://medium.com/sipwines/the-abcs-of-sustai...,['Sip Wines'],2021-03-02 23:39:49.948000+00:00,"['Wine Tasting', 'Sustainability', 'Wine']"


We will use the article title and its text for generating embeddings. For that, we join the article title and the first 1000 characters from the article text.

In [36]:
# select first 1000 characters
df["text"] = df["text"].str[:1000]
# join article title and the text
df["title_text"] = df["title"] + ". " + df["text"]

# Initialize NER Model

To extract named entities, we will use a NER model finetuned on a BERT-base model. The model can be loaded from the HuggingFace model hub as follows:

In [37]:
import torch

# set device to GPU if available
cuda_avail =  torch.cuda.is_available()
print(f"Cuda? {cuda_avail}")
device = torch.cuda.current_device() if torch.cuda.is_available() else None
print(f"Device: {device}")

Cuda? True
Device: 0


In [38]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

model_id = "dslim/bert-base-NER"

# load the tokenizer from huggingface
tokenizer = AutoTokenizer.from_pretrained(
    model_id
)
# load the NER model from huggingface
model = AutoModelForTokenClassification.from_pretrained(
    model_id
)
# load the tokenizer and model into a NER pipeline
nlp = pipeline(
    "ner",
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="max",
    device=device
)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: CUDA error: unknown error
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [20]:
text = "London is the capital of England and the United Kingdom"
# use the NER pipeline to extract named entities from the text
nlp(text)

[{'entity_group': 'LOC',
  'score': 0.9996493,
  'word': 'London',
  'start': 0,
  'end': 6},
 {'entity_group': 'LOC',
  'score': 0.9997588,
  'word': 'England',
  'start': 25,
  'end': 32},
 {'entity_group': 'LOC',
  'score': 0.9993923,
  'word': 'United Kingdom',
  'start': 41,
  'end': 55}]

Our NER pipeline is working as expected and accurately extracting entities from the text.

# Initialize Retriever

A retriever model is used to embed passages (article title + first 1000 characters) and queries. It creates embeddings such that queries and passages with similar meanings are close in the vector space. We will use a sentence-transformer model as our retriever. The model can be loaded as follows:

In [21]:
from sentence_transformers import SentenceTransformer

# load the model from huggingface
retriever = SentenceTransformer(
    'flax-sentence-embeddings/all_datasets_v3_mpnet-base',
    device=device
)
retriever

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

# Initialize Pinecone Index

Now we need to initialize our Pinecone index. The Pinecone index stores vector representations of our passages which we can retrieve using another vector (the query vector). We first need to initialize our connection to Pinecone. For this, we need a free [API key](https://app.pinecone.io/); you can find your environment in the [Pinecone console](https://app.pinecone.io) under **API Keys**. We initialize the connection like so:

Now we setup our index specification, this allows us to define the cloud provider and region where we want to deploy our index. You can find a list of all [available providers and regions here](https://docs.pinecone.io/docs/projects).

In [22]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

Now we can create our vector index. We will name it `ner-search` (feel free to chose any name you prefer). We specify the metric type as `cosine` and dimension as `768` as these are the vector space and dimensionality of the vectors output by the retriever model.

In [23]:
index_name = "ner-search"

In [25]:
import time

# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    print(f"Create index: {index_name}")
    pc.create_index(
        index_name,
        dimension=768,
        metric='cosine',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        print(f"\tsleeing, waiting for {inindex_name} to be ready")
        time.sleep(1)

Create index: ner-search


In [26]:
# connect to index
index = pc.Index(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

# Generate Embeddings and Upsert

We generate embeddings for the `title_text` column we created earlier. Alongside the embeddings, we also include the named entities in the index as metadata. Later we will apply a filter based on these named entities when executing queries.

Let's first write a helper function to extract named entities from a batch of text.

In [27]:
def extract_named_entities(text_batch):
    # extract named entities using the NER pipeline
    extracted_batch = nlp(text_batch)
    entities = []
    # loop through the results and only select the entity names
    for text in extracted_batch:
        ne = [entity["word"] for entity in text]
        entities.append(ne)
    return entities

Now we create the embeddings. We do this in batches of `64` to avoid overwhelming machine resources or API request limits.

In [28]:
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

# we will use batches of 64
batch_size = 64

for i in tqdm(range(0, len(df), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(df))
    # extract batch
    batch = df.iloc[i:i_end].copy()
    # generate embeddings for batch
    emb = retriever.encode(batch["title_text"].tolist()).tolist()
    # extract named entities from the batch
    entities = extract_named_entities(batch["title_text"].tolist())
    # remove duplicate entities from each record
    batch["named_entities"] = [list(set(entity)) for entity in entities]
    batch = batch.drop('title_text', axis=1)
    # get metadata
    meta = batch.to_dict(orient="records")
    # create unique IDs
    ids = [f"{idx}" for idx in range(i, i_end)]
    # add all to upsert list
    to_upsert = list(zip(ids, emb, meta))
    # upsert/insert these records to pinecone
    _ = index.upsert(vectors=to_upsert)

# check that we have all vectors in index
index.describe_index_stats()

  0%|          | 0/313 [00:00<?, ?it/s]

--- Logging error ---
Traceback (most recent call last):
  File "/home/sean/miniforge3/envs/pinecode-haystack-env/lib/python3.10/logging/__init__.py", line 1098, in emit
    msg = self.format(record)
  File "/home/sean/miniforge3/envs/pinecode-haystack-env/lib/python3.10/logging/__init__.py", line 942, in format
    return fmt.format(record)
  File "/home/sean/miniforge3/envs/pinecode-haystack-env/lib/python3.10/logging/__init__.py", line 678, in format
    record.message = record.getMessage()
  File "/home/sean/miniforge3/envs/pinecode-haystack-env/lib/python3.10/logging/__init__.py", line 368, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/home/sean/miniforge3/envs/pinecode-haystack-env/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/sean/miniforge3/envs/pinecode-haystack-env/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 19968}},
 'total_vector_count': 19968}

Now we have indexed the articles and relevant metadata. We can move on to querying.

# Querying

First, we will write a helper function to handle the queries.

In [29]:
from pprint import pprint

def search_pinecone(query):
    # extract named entities from the query
    ne = extract_named_entities([query])[0]
    # create embeddings for the query
    xq = retriever.encode(query).tolist()
    # query the pinecone index while applying named entity filter
    xc = index.query(xq, top_k=10, include_metadata=True, filter={"named_entities": {"$in": ne}})
    # extract article titles from the search result
    r = [x["metadata"]["title"] for x in xc["matches"]]
    return pprint({"Extracted Named Entities": ne, "Result": r})

Now try a query.

In [30]:
query = "What are the best places to visit in Greece?"
search_pinecone(query)

ValueError: The argument order for `query()` has changed; please use keyword arguments instead of positional arguments. Example: index.query(vector=[0.1, 0.2, 0.3], top_k=10, namespace='my_namespace')

In [ ]:
query = "What are the best places to visit in London?"
search_pinecone(query)

In [ ]:
query = "Why does SpaceX want to build a city on Mars?"
search_pinecone(query)

These all look like great results, making the most of Pinecone's advanced vector search capabilities while limiting search scope to relevant records only with a named entity filter.

In [ ]:
pc.delete_index(index_name)